# Notebook for preprocessing Wikipedia (English) dataset

### Initilizing phonemizer and tokenizer

In [3]:
import yaml

config_path = "Configs/config.yml" # you can change it to anything else
config = yaml.safe_load(open(config_path))

In [4]:
from phonemize import phonemize

In [5]:
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained(config['dataset_params']['tokenizer'])

### Process dataset

In [ ]:
import datasets
import pathlib

datasets.config.DOWNLOADED_DATASETS_PATH = pathlib.Path("/media/yamauchi/2084DFA884DF7EAA/dataset/wikipedia-ja")


dataset = datasets.load_dataset(
    'wikipedia', language="ja", date="20210120", beam_runner="DirectRunner",
    cache_dir="/media/yamauchi/2084DFA884DF7EAA/dataset/wikipedia-ja/.cache"
)
dataset = dataset['train']

# datasets/datasets/wikipedia/wikipedia.py
# comment out Line 523
# | "Distribute" >> beam.transforms.Reshuffle() 

# dataset = datasets.load_dataset(
#     "./wikipedia.py", language="ja", date="20210120", beam_runner="DirectRunner",
#     cache_dir="/media/yamauchi/2084DFA884DF7EAA/dataset/wikipedia-ja/.cache"
# )
# dataset = dataset['train']

print(len(dataset))
print(dataset[0])

In [ ]:
root_directory = "./wiki_phoneme" # set up root directory for multiprocessor processing

In [ ]:
import os
num_shards = 50000

def process_shard(i):
    directory = root_directory + "/shard_" + str(i)
    if os.path.exists(directory):
        print("Shard %d already exists!" % i)
        return
    print('Processing shard %d ...' % i)
    shard = dataset.shard(num_shards=num_shards, index=i)
    processed_dataset = shard.map(lambda t: phonemize(t['text'], tokenizer), remove_columns=['text'])
    if not os.path.exists(directory):
        os.makedirs(directory)
    processed_dataset.save_to_disk(directory)

In [ ]:
from pebble import ProcessPool
from concurrent.futures import TimeoutError

#### Note: You will need to run the following cell multiple times to process all shards because some will fail. Depending on how fast you process each shard, you will need to change the timeout to a longer value to make more shards processed before being killed.


In [ ]:
max_workers = 32 # change this to the number of CPU cores your machine has 

with ProcessPool(max_workers=max_workers) as pool:
    pool.map(process_shard, range(num_shards), timeout=60)

### Collect all shards to form the processed dataset

In [ ]:
from datasets import load_from_disk, concatenate_datasets

output = [dI for dI in os.listdir(root_directory) if os.path.isdir(os.path.join(root_directory,dI))]
datasets = []
for o in output:
    directory = root_directory + "/" + o
    try:
        shard = load_from_disk(directory)
        datasets.append(shard)
        print("%s loaded" % o)
    except:
        continue

In [ ]:
dataset = concatenate_datasets(datasets)
dataset.save_to_disk(config['data_folder'])
print('Dataset saved to %s' % config['data_folder'])

In [ ]:
# check the dataset size
dataset

### Remove unneccessary tokens from the pre-trained tokenizer
The pre-trained tokenizer contains a lot of tokens that are not used in our dataset, so we need to remove these tokens. We also want to predict the word in lower cases because cases do not matter that much for TTS. Pruning the tokenizer is much faster than training a new tokenizer from scratch. 

In [ ]:
from datasets import load_from_disk
from simple_loader import FilePathDataset, build_dataloader

dataset = load_from_disk(config['data_folder'])
file_data = FilePathDataset(dataset)
loader = build_dataloader(file_data, num_workers=32, batch_size=128)

In [ ]:
special_token = config['dataset_params']['word_separator']

In [ ]:
# get all unique tokens in the entire dataset

from tqdm import tqdm

unique_index = [special_token]
for _, batch in enumerate(tqdm(loader)):
    unique_index.extend(batch)
    unique_index = list(set(unique_index))

In [ ]:
# redo the mapping for lower number of tokens

token_maps = {}
for t in tqdm(unique_index):
    word = tokenizer.decode([t])
    token_maps[t] = {'word': word, 'token': unique_index.index(t)}

In [ ]:
import pickle
with open(config['dataset_params']['token_maps'], 'wb') as handle:
    pickle.dump(token_maps, handle)
print('Token mapper saved to %s' % config['dataset_params']['token_maps'])

### Test the dataset with dataloader


In [ ]:
from dataloader import build_dataloader

train_loader = build_dataloader(dataset, batch_size=32, num_workers=0, dataset_config=config['dataset_params'])

In [ ]:
print(len(train_loader))

In [ ]:
_, (words, labels, phonemes, input_lengths, masked_indices) = next(enumerate(train_loader))